In [1]:
%pip install spacy
%pip install vaderSentiment
%pip install pyarrow
%pip install torch
%pip install nltk
%pip install textblob
%pip install -U textblob-de
%pip install transformers
!{sys.executable} -m spacy download de_core_news_lg
!{sys.executable} -m spacy download de_core_news_md

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: parse error near `-m'
zsh:1: parse error near `-m'


In [5]:
!{sys.executable} -m spacy download de_core_news_md

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/de_core_news_md-2.3.0/de_core_news_md-2.3.0.tar.gz#egg=de_core_news_md==2.3.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/47.0 MB 626.2 kB/s eta 0:01:12^C
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/47.0 MB 623.0 kB/s eta 0:01:12
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File 

In [2]:
import os
import re
import pandas as pd
from pyarrow import feather
import numpy as np

#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import sys
sys.path.append("GERVader")
from vaderSentimentGER import SentimentIntensityAnalyzer # -- https://github.com/KarstenAMF/GerVADER
from nltk.tag import pos_tag
import sys
import nltk
#nltk.download("vader_lexicon")

from scipy.special import softmax
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

# GERVader

In [11]:
pd.set_option('display.max_colwidth', None)
post_data = pd.read_feather('reddit_res/posts.ftr')
comment_data = pd.read_feather('reddit_res/comments.ftr')
post_data

,title,text,subreddit,created
0,25€ postgebühr aliexpress sendungen,momentan 25 gaudigebhr post bestellungen aliexpress voriges stndig 25 post bezahlen bestellwerten wenigen euro wirtschaftlich pakete teilweise angenommen absolute resourcenverschwendung umweltverschmutzung vertrottelte verpackungsverordung ohnehin vorerst mittlerweile sachen aliexpress zwischenzeit bruchte bauteile wrde ichs gerne probierten lust stndig 25 zahlen vat entrichtet gebhr versandart umgehen,austria,2023-05-22 11:28:50
1,frage nachbarländer einkaufen deutlich günstiger klimaticket erreichbar,inspiriert post deutschen deutschlandticket regelmig polen einkaufen fahrt gnstiger preuen frag nachbarn deutlich gnstiger einkaufen deutschland gnstiger sterreich passau klimaticket erreichbar tschechien slowakei ungarn slowenien gnstiger gibts orte gnstiger sterreich einkaufen klimaticket gilt italien schweiz teurer wren wien hinzufahren euro sparen rest 3h spartipps,austria,2023-05-22 09:29:17
3,klimabonus,werte mitbrgerinnen klimabonus fr 2022 4 kontakt zustndigen stelle vertrstet 1 versucht altes bankkonto berweisen 2021 aktualisiert steuerausgleich aufs konto bekommen korrigiert 23 wochen warten 2 kontakt herr berweisung fehlgeschlagen 2tes berwiesen bekomme sodfxo gutscheine zugeschickt 34 wochen dauern 3 kontakt anfang mrz vertrstet nchste welle aussendungen 34wochen warten 4 kontakt 1 woche urlaub spital aufenthalt formular warte antwort bereit idioten zustehende geld schenken,austria,2023-05-22 00:18:07
4,klima service lucky car erfahrungen,wrde gerne klimaservice lucky car nhe wien umgebung aufgemacht lucky car nachfragen leistung zufrieden seids gibts preislich standort bezogene unterschiede wrd erfahrungen hren,austria,2023-05-21 14:05:15
5,flughafen wien wasser warm/heiß,wasser wasserhhnen toiletten flughafen wien warm hei raus heizen wasser fr 4 mineralwasser kauft woke umweltbewusst,austria,2023-05-20 17:05:23
6,amal,ahnung klimakatastrophe retten knnen klammern hoffnung wundertechnik klammern irrglauben htten technik wren politiker korrupt einzusetzen ignorieren thema x200b ists logisch heutigen system konsumismus zwang immerwhrenden wirtschaftswachstums klimaproblem gelst knnen eautos grte greenwashing generation neuerdings geliebten akws helfen energie berfluss wege verbrauchen super server nutzlose kalkulationen fr diverse blockchains bitcoin miner super server millionen daten speichern exponentiell anhufen straen infrastruktur fr autos grten verschmutzer berhaupt idee autos wegkommen wirtschaft hngt bauen lsung eautos beschlieen 2035 verbrenner straen plan stromnetz jemals verkraften berhaupt bentigte energie stammen deutschland vorreiter erneuerbaren energie grad grad geschafft 55 energie stromsektor erneuerbare ersetzen stromsektor 20 gesamtenergieverbrauchs landes x200b gezwungen vereinbarung ewigem wirtschaftswachstum klimaschutz wohlstand gesellschaft hngt drogenschtiger heroin wrde gesamte konstrukt auseinanderbrechen wrden wohlstand verlieren armes landarme privileg bers klima sorgen,austria,2023-05-20 11:13:41
7,klimakleben entlassen,wrde festkleben chef entlassen wrde natrlich freizeit,austria,2023-05-17 18:46:22
10,ö3-jugendstudie gen heiraten lehrpläne klimapolitik,92 prozent mchten einfamilienhaus eigentumswohnung leben hahahahhahah grnen bricht jugend schei freu unermessliche scheinbar junge generation deppat,austria,2023-05-16 07:52:50
11,fpö-wählern leben schwer,auslnder nato impfungen radfahrer mchtigen flchtende kinder klimaaktivisten binneni veganer usa wissenschaft sozialdemokraten greta thunberg ukraine nichtraucherzonen orf tempolimits bio qualittsjournalismus frauen auslnder begegnungszonen grtelpools eu fremdsprachen grnen fleischersatzprodukte bildung registrierkassen justiz grnflchen eliten armin wolf lgbtq wort tchter x200b gefunden twitter meinung,austria,2023-05-15 17:45:48
14,mikl-leitner nennt kooperation fpö reibungslos schließt koalition kickl,niedersterreichs landeshauptfrau bilanz ber monate regierungsarbeit

In [4]:
def perform_sentiment_analysis_vader(data):
    data_temp = data.copy()
    sid = SentimentIntensityAnalyzer()
    data_temp["Negative"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["neg"])
    data_temp["Positive"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["pos"])
    data_temp["Neutral"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["neu"])
    data_temp["Compound"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["compound"])
    data_temp["Overall"] = ['Positive' if score >= 0.05 else 'Negative' if score <= -0.05 else 'Neutral' for score in data_temp["Compound"]]
    return data_temp

In [5]:
post_data_vader = perform_sentiment_analysis_vader(post_data)
comment_data_vader = perform_sentiment_analysis_vader(comment_data)

In [6]:
overall_sentiments_vader = pd.concat([post_data_vader["Overall"].value_counts(), comment_data_vader["Overall"].value_counts()], axis=1)
overall_sentiments_vader.columns = ["Post Data Sentiment", "Comment Data Sentiment"]
overall_sentiments_vader

,Post Data Sentiment,Comment Data Sentiment
Positive,41,3730
Negative,8,1897
Neutral,7,1802


# TextblobDE

In [7]:
from textblob_de import TextBlobDE

In [8]:
def perform_sentiment_analysis_blob(data):
    data_temp = data.copy()
    data_temp["Polarity"] = data_temp.apply(lambda x: TextBlobDE(x["text"]).sentiment.polarity, axis=1)
    data_temp["Subjectivity"] = data_temp.apply(lambda x: TextBlobDE(x["text"]).sentiment.subjectivity, axis=1)
    data_temp["Overall"] = ['Negative' if score < 0.0 else 'Neutral' if score == 0.0 else 'Positive' for score in data_temp["Polarity"]]
    return data_temp

In [9]:
post_data_textblob = perform_sentiment_analysis_blob(post_data)
comment_data_textblob = perform_sentiment_analysis_blob(comment_data)

In [10]:
overall_sentiments_blob = pd.concat([post_data_textblob["Overall"].value_counts(), comment_data_textblob["Overall"].value_counts()], axis=1)
overall_sentiments_blob.columns = ["Post Data Sentiment", "Comment Data Sentiment"]

In [11]:
overall_sentiments_blob

,Post Data Sentiment,Comment Data Sentiment
Positive,31,2769
Neutral,17,3166
Negative,8,1494


# Pre-trained GeRiBERTa model

In [12]:
import spacy
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'oliverguhr/german-sentiment-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/opt/homebrew/Caskroom/miniforge/base/envs/kddm1HW1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
text_list = [post_data["text"]]


[0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [4]:
import spacy
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'oliverguhr/german-sentiment-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def perform_sentiment_analysis(text):
    # Tokenize and truncate the text
    tokens = tokenizer.tokenize(text)
    max_length = tokenizer.model_max_length  # Maximum sequence length for the model
    tokens = tokens[:max_length-2]  # Account for special tokens [CLS] and [SEP]
    tokens = ['[CLS]'] + tokens + ['[SEP]']

    # Convert tokens to input IDs
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    # Feed the tokenized input to the pre-trained model
    with torch.no_grad():
        logits = model(input_ids)[0]

    scores = torch.softmax(logits, dim=1)
    sentiment_scores = scores.tolist()[0]
    rounded_scores = [round(score, 4) for score in sentiment_scores]
    
    # Get the predicted sentiment label
    predicted_label = logits.argmax().item()

    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[predicted_label]

    return sentiment, rounded_scores

nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

post_data_geriberta = post_data.copy()
comment_data_geriberta = comment_data.copy()

post_data_geriberta["preprocessed_text"] = post_data_geriberta["text"].apply(preprocess_text)
post_data_geriberta[["Sentiment", "Scores"]] = post_data_geriberta["preprocessed_text"].apply(perform_sentiment_analysis).apply(pd.Series)
post_data_geriberta["Sentiment"] = post_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())

comment_data_geriberta["preprocessed_text"] = comment_data_geriberta["text"].apply(preprocess_text)
comment_data_geriberta[["Sentiment", "Scores"]] = comment_data_geriberta["preprocessed_text"].apply(perform_sentiment_analysis).apply(pd.Series)
comment_data_geriberta["Sentiment"] = comment_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())


OSError: [E050] Can't find model 'de_core_news_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [15]:
post_data_geriberta = post_data.copy()
comment_data_geriberta = comment_data.copy()

post_data_geriberta[["Sentiment", "Scores"]] = post_data_geriberta["text"].apply(perform_sentiment_analysis).apply(pd.Series)
post_data_geriberta["Sentiment"] = post_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())

comment_data_geriberta[["Sentiment", "Scores"]] = comment_data_geriberta["text"].apply(perform_sentiment_analysis).apply(pd.Series)
comment_data_geriberta["Sentiment"] = comment_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())

In [16]:
overall_sentiments_geriberta = pd.concat([post_data_geriberta["Sentiment"].value_counts(), comment_data_geriberta["Sentiment"].value_counts()], axis=1)
overall_sentiments_geriberta.columns = ["Post Data Sentiment", "Comment Data Sentiment"]

In [1]:
overall_sentiments_geriberta 

NameError: name 'post_data_geriberta' is not defined

In [18]:
from nltk.tokenize import word_tokenize
from collections import Counter

word_counts = Counter()
for corpus in post_data_geriberta["text"]:
    words = word_tokenize(corpus)
    word_counts.update(words)

sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
print("Most frequently used sentiment words:")
for word, count in sorted_words:
    print(word, ":", count)

Most frequently used sentiment words:
* : 136
mal : 32
ja : 29
i : 28
gibt : 27
schon : 27
immer : 24
mehr : 21
\ : 20
to : 19
linz : 18
klimaticket : 17
einfach : 15
wien : 15
. : 15
the : 15
jemand : 12
menschen : 12
is : 12
warum : 12
beim : 11
a : 11
studium : 11
“ : 11
russland : 11
österreich : 10
gut : 10
überhaupt : 10
leute : 10
wirklich : 10
and : 10
seit : 10
stadt : 10
lassen : 9
thema : 9
wäre : 9
for : 9
gemacht : 9
wer : 9
” : 9
wurde : 8
vielleicht : 8
wohl : 8
fragen : 8
stellen : 8
zeit : 8
graz : 8
'm : 8
danke : 8
's : 7
paar : 7
nie : 7
bzw : 7
& : 7
eigentlich : 7
radfahrer : 7
ukraine : 7
spö : 7
fahren : 7
sowieso : 7
it : 7
ticket : 7
radweg : 7
post : 6
gerne : 6
ab : 6
hallo : 6
erhalten : 6
1 : 6
anfang : 6
dabei : 6
trotzdem : 6
erfahrungen : 6
# : 6
x200b : 6
ganz : 6
daran : 6
macht : 6
fast : 6
leben : 6
usa : 6
meinung : 6
– : 6
land : 6
regierung : 6
politik : 6
ned : 6
be : 6
what : 6
komplett : 6
günstiger : 5
gehen : 5
kommt : 5
deutschland : 5
weit

In [19]:
with open("stopwords/stopwords-de.txt", "r") as file:
    additional_stop_words = set(file.read().splitlines())
    
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(post_data_vader['text'])

def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
print_topics(lda, count_vectorizer, n_top_words)

NameError: name 'lda' is not defined